<a href="https://colab.research.google.com/github/sreekar-pasumarthi/Deep-Learning-Projects/blob/master/glass_stackedautoclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#stacked autoencoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
df_train = pd.read_csv('https://raw.githubusercontent.com/sreekar-pasumarthi/Deep-Learning-Projects/master/glass/glass.csv')
print(f'The train set contain {df_train.shape[0]} examples')
print(f'The train set contain {df_train.shape[1]} features')
df_train.head()#to select first n rows

The train set contain 214 examples
The train set contain 10 features


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [0]:
X_train = df_train.drop('Type', axis = 1)#drops the column type from the dataframe
y_train = df_train['Type']

In [0]:
X_train.describe()

In [11]:
X_train=((X_train-X_train.mean())/(X_train.max()-X_train.min()))
X_train.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
count,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02
mean,3.542954e-14,-5.429847e-16,-2.116687e-16,-9.480993e-17,1.303474e-16,3.086835e-17,-4.302763e-17,1.188042e-16,1.530448e-17
std,1.333127e-01,1.227975e-01,3.212490e-01,1.555357e-01,1.383117e-01,1.050228e-01,1.322633e-01,1.578474e-01,1.910563e-01
min,-3.167437e-01,-4.026843e-01,-5.978915e-01,-3.597840e-01,-5.073097e-01,-8.004124e-02,-3.277846e-01,-5.557039e-02,-1.117830e-01
25%,-8.090081e-02,-7.524067e-02,-1.268447e-01,-7.941014e-02,-6.623832e-02,-6.031499e-02,-6.663221e-02,-5.557039e-02,-1.117830e-01
50%,-3.008870e-02,-1.621812e-02,1.771642e-01,-2.645064e-02,2.483311e-02,9.330745e-03,-3.317496e-02,-5.557039e-02,-1.117830e-01
75%,3.477083e-02,6.272925e-02,2.038903e-01,5.766151e-02,7.795811e-02,1.818743e-02,2.003136e-02,-5.557039e-02,8.429540e-02
max,6.832563e-01,5.973157e-01,4.021085e-01,6.402160e-01,4.926903e-01,9.199588e-01,6.722154e-01,9.444296e-01,8.882170e-01


In [0]:
X_train.head()

In [0]:
X_train.shape

In [7]:
from keras.layers import Input, Dense
from keras.models import Model

input_img = Input(shape=(9,))
activation_fn= 'tanh'
epochs=200
batch_size=1
shuffle=True

encoded = Dense(5, activation= activation_fn)(input_img)
decoded = Dense(9, activation='tanh')(encoded)

#9->5->9
# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 9)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 50        
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 54        
Total params: 104
Trainable params: 104
Non-trainable params: 0
_________________________________________________________________


In [0]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')
autoencoder.fit(X_train, X_train,epochs=epochs,batch_size=batch_size,shuffle=shuffle)

In [16]:
from keras.layers import Input, Dense
from keras.models import Model

batch_size = 1
epochs = 100

# Layer-wise pretraining
encoders = []
nb_layers = [9, 5, 4]
X_train_tmp = np.copy(X_train)
for i, (n_in, n_out) in enumerate(zip(nb_layers[:-1], nb_layers[1:]), start=1):
    print('Training the layer {}: Input {} -> Output {}'.format(i, n_in, n_out))
    # Create AE and training
    input_img = Input(shape=(n_in,))
    encoded = Dense(n_out, activation='sigmoid')(input_img)
    decoded = Dense(n_in, activation='sigmoid')(encoded)
    autoencoder = Model(input_img, decoded)
    encoder=Model(input_img,encoded)
    autoencoder.compile(loss='mean_squared_error', optimizer='rmsprop')
    autoencoder.fit(X_train_tmp, X_train_tmp, batch_size=batch_size, epochs=epochs)

    # Store trained weights and update training data
    encoders.append(autoencoder.layers[0].get_weights())
    X_train_tmp = encoder.predict(X_train_tmp)

Training the layer 1: Input 9 -> Output 5
Epoch 1/100
214/214 [==============================] - 0s 932us/step - loss: 0.2102
Epoch 2/100
214/214 [==============================] - 0s 812us/step - loss: 0.1083
Epoch 3/100
214/214 [==============================] - 0s 813us/step - loss: 0.0583
Epoch 4/100
214/214 [==============================] - 0s 777us/step - loss: 0.0397
Epoch 5/100
214/214 [==============================] - 0s 823us/step - loss: 0.0335
Epoch 6/100
214/214 [==============================] - 0s 812us/step - loss: 0.0314
Epoch 7/100
214/214 [==============================] - 0s 846us/step - loss: 0.0306
Epoch 8/100
214/214 [==============================] - 0s 796us/step - loss: 0.0302
Epoch 9/100
214/214 [==============================] - 0s 800us/step - loss: 0.0300
Epoch 10/100
214/214 [==============================] - 0s 809us/step - loss: 0.0298
Epoch 11/100
214/214 [==============================] - 0s 783us/step - loss: 0.0297
Epoch 12/100
214/214 [==========

In [0]:
#classification after training

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense

In [18]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)#to convert y_train into to 'one hot vector'
y_train.shape

(214, 8)

In [0]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(10,	input_dim=4,	activation='relu'))
	model.add(Dense(8,	activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [21]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=1, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator,X_train_tmp, y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 48.16% (4.66%)
